<a href="https://colab.research.google.com/github/TeamBJEJ/Natural-Language-Processing-with-Disaster-tweet/blob/main/NLP_with_disaster_glove_embedding_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect Kaggle

In [1]:
!!pip install kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
ls -1ha kaggle.json

kaggle.json


In [ ]:
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes        110           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        216           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        615           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       3633           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      29385           False  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge       7633           False  
connectx

In [3]:
!kaggle competitions download -c nlp-getting-started

  0% 0.00/411k [00:00<?, ?B/s]
100% 411k/411k [00:00<00:00, 55.6MB/s]
  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 63.2MB/s]
  0% 0.00/22.2k [00:00<?, ?B/s]
100% 22.2k/22.2k [00:00<00:00, 23.0MB/s]


# Import libraries

In [37]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Input, Activation, Dropout, LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
numpy.random.seed(7)

In [5]:
ls

drive/  kaggle.json  sample_data/  sample_submission.csv  test.csv  train.csv


In [6]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2021-04-11 07:33:23--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-04-11 07:33:23--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-04-11 07:33:24--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

# Preparing data

In [7]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [8]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
X = train_data.drop(['target'], axis = 1)
y = train_data['target']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [11]:
print(len(X_train))
print(len(X_test))

6090
1523


In [12]:
X_train_text = X_train['text']
X_test_text = X_test['text']
X_test_text.head()

3228    Goulburn man Henry Van Bilsen missing: Emergen...
3706    The things we fear most in organizations--fluc...
6957                              @tsunami_esh ?? hey Esh
2887    @POTUS you until you drown by water entering t...
7464    Crawling in my skin\nThese wounds they will no...
Name: text, dtype: object

# Glove Embedding 

In [13]:
t = Tokenizer()
t.fit_on_texts(X_train_text)
vocab_size = len(t.word_index) + 1
print(vocab_size)

19424


In [16]:
embedding_dict = dict()
f = open('glove.6B.100d.txt', encoding='utf-8')

for line in f:
    word_vector = line.split()
    word = word_vector[0]
    word_vector_arr = np.asarray(word_vector[1:], dtype = 'float32')
    embedding_dict[word] = word_vector_arr

f.close()
print("%s number of embedding vector" % len(embedding_dict))

400000 number of embedding vector


In [17]:
embedding_matrix = np.zeros((vocab_size, 100))
np.shape(embedding_matrix)

(19424, 100)

In [27]:
cnt = 0
exist = 0
for word, i in t.word_index.items():
    temp = embedding_dict.get(word)
    if temp is not None:
        embedding_matrix[i] = temp
        exist += 1
    else:
        cnt += 1
print(cnt, exist)

8289 11134


In [21]:
X_train_encoded = t.texts_to_sequences(X_train_text)
X_test_encoded = t.texts_to_sequences(X_test_text)

In [22]:
max_len = max(len(l) for l in X_train_encoded)
max_len

33

In [32]:
X_train_encoded = sequence.pad_sequences(X_train_encoded, maxlen = max_len, padding = 'post')
X_test_encoded = sequence.pad_sequences(X_test_encoded, maxlen = max_len, padding = 'post')

In [31]:
embedding_vocab_length = 100
embedding_layer = Embedding(vocab_size, embedding_vocab_length, weights = [embedding_matrix], \
input_length=max_len, trainable = False)

In [39]:
X_indices = Input(max_len)
embeddings = embedding_layer(X_indices)
X = LSTM(128, return_sequences = True)(embeddings)
X = Dropout(0.6)(X)
X = LSTM(128, return_sequences=True)(X)
X = Dropout(0.6)(X)
X = Dense(1, activation = 'sigmoid')(X)
model = Model(inputs = X_indices, outputs = X)


In [40]:
adam = keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer = adam, loss = 'binary_crossentropy', metrics=['accuracy'])
print(model.summary())
model.fit(X_train_encoded, y_train, validation_data=(X_test_encoded, y_test), epochs=15, batch_size=64)
scores = model.evaluate(X_test_encoded, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 33)]              0         
_________________________________________________________________
embedding_3 (Embedding)      multiple                  1942400   
_________________________________________________________________
lstm_5 (LSTM)                (None, 33, 128)           117248    
_________________________________________________________________
dropout_2 (Dropout)          (None, 33, 128)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 33, 128)           131584    
_________________________________________________________________
dropout_3 (Dropout)          (None, 33, 128)           0         
_________________________________________________________________
dense_4 (Dense)              (None, 33, 1)             129 